In [410]:
import sys
sys.path.append('../../apple-model-app/')

from helpers import *
import pandas as pd
import json

import dash_bootstrap_components as dbc
import dash_html_components as html

pd.set_option('max_column', 0)

def get_logo(team, size=60):

    from unidecode import unidecode
    import json

    # load team logos
    teams = json.load(open('../../apple-model-app/assets/team-styles.json'))

    try:
        logo = teams[unidecode(team)]['logo']

    except:
        logo = None 

    return logo

def get_logo_dims(team, size=60):

    from unidecode import unidecode
    import json

    # load team logos
    teams = json.load(open('../../apple-model-app/assets/team-styles.json'))

    try:
        logo = teams[unidecode(team)]['logo']
        _, (width, height) = getsizes(logo)
        
        if  height >= width:
            width = (width / height) * size
            height = (height / height) * size
        else:
            height = (height / width) * size
            width = (width / width) * size    
    except:
        logo = None 
        width = size
        height = size

    return {'height' : f"{height}px", 'width' : f"{width}px"}

def get_forward_ids(team_roster):

    forwards = team_roster[team_roster.position == 'F']

    left_wing = [f['playerid'] for f in forwards[fields].to_dict('records') if (f['position_count'] == 1) & ('LW' in f['positions'])]
    centers = [f['playerid'] for f in forwards[fields].to_dict('records') if (f['position_count'] == 1) & ('C' in f['positions'])]
    right_wing = [f['playerid'] for f in forwards[fields].to_dict('records') if (f['position_count'] == 1) & ('RW' in f['positions'])]

    centers = centers[:5]
    left_wing = left_wing[:5]
    right_wing = right_wing[:5]

    centers += [f['playerid'] for f in forwards[fields].to_dict('records') \
                if (f['position_count'] == 2)\
                & ('C' in f['positions'])\
                & (f['playerid'] not in centers + left_wing + right_wing)]

    centers = centers[:5]

    left_wing += [f['playerid'] for f in forwards[fields].to_dict('records')\
                  if (f['position_count'] == 2)\
                  & ('LW' in f['positions'])\
                  & (f['playerid'] not in centers + left_wing + right_wing)][:5]

    left_wing = left_wing[:5]

    right_wing += [f['playerid'] for f in forwards[fields].to_dict('records')\
                   if (f['position_count'] == 2)
                   & ('RW' in f['positions'])\
                   & (f['playerid'] not in centers + left_wing + right_wing)][:5]

    right_wing = right_wing[:5]

    remaining = forwards[~forwards.playerid.isin(left_wing + centers + right_wing)].playerid.tolist()

    while len(left_wing) < 5:
        if remaining:
            left_wing.append(remaining.pop())
        else:
            break

    while len(centers) < 5:
        if remaining:
            centers.append(remaining.pop())
        else:
            break

    while len(right_wing) < 5:
        if remaining:
            right_wing.append(remaining.pop())
        else:
            break
            
    return left_wing, centers, right_wing

def get_defence_ids(team_roster):

    defence = team_roster[team_roster.position == 'D']

    left_d = [d['playerid'] for d in defence[fields].to_dict('records') \
                if (d['position_count'] == 1)\
                & ('D' in d['positions'])\
                & (d['shoots'] == 'L')][:4]

    right_d = [d['playerid'] for d in defence[fields].to_dict('records') \
                if (d['position_count'] == 1)\
                & ('D' in d['positions'])\
                & (d['shoots'] == 'R')][:4]

    remaining = defence[~defence.playerid.isin(right_d + left_d)].playerid.tolist()

    while len(left_d) < 4:
        if remaining:
            left_d.append(remaining.pop())
        else:
            break

    while len(right_d) < 4:
        if remaining:
            right_d.append(remaining.pop())
        else:
            break
            
    return left_d, right_d

def get_roster_card(playerid):
    
    player = rosters[rosters.playerid == int(playerid)]
    
    try:
        flag = player.nation_flag.item()
        logo = get_logo(player.rights.item())
        dims = get_logo_dims(player.rights.item(), 20)
    except:
        flag = player.nation_flag.item()
        logo = None 

    card = dbc.Card(
      [
          dbc.CardHeader(
            [
              dbc.Row(
                [
                dbc.Col(
                  html.Img(src=logo, **dims) if logo else None,
                      width=2,
                      align='start',
              ),
              dbc.Col(
                  html.P(f'''{player.player_name.item()} {flag}'''
                              ),
                              width=10,
                              align="center", 
                      )
              ],
              no_gutters=True
              ),
            ],
          style={'font-size' : '1vw', 'height' : '30%', 'width' : '100%','font-weight' : 'bold'}
          ),
          dbc.CardBody(
              dbc.Row(
                [
                  dbc.Col([
                      html.P('NHL %'),
                      html.P('NHL Role'),
                      html.P('Round'),
                      html.P('Pick'),
                  ],
                  ),
                  dbc.Col([
                      html.P(f'{str(int(player.nhl_likelihood.item() * 100)) + "%" if ~np.isnan(player.nhl_likelihood.item()) else "N/A"}'),
                      html.P(player['Projected Role'].item() if ~np.isnan(player.nhl_likelihood.item()) else "N/A"),
                      html.P(player.draft_round.item() if ~np.isnan(player.draft_round.item()) else "N/A"),
                      html.P(player.draft_pick.item() if ~np.isnan(player.draft_pick.item()) else "N/A"),
                  ],
                  style={'font-weight' : 'bold'}
                  )
              ],
              style={'font-size' : '0.6vw', 'width' : '100%', 'height' : '70%'},
              no_gutters=True
            )
          ),
      ],
      style={"width": "100%", 'height' : '96%', 'textAlign': 'center', 'justify-content' : 'center', 'vertical-align' : 'middle'}
    )
    
    return card

countries = {
            'USA' : 'USA',
            'AUS' : 'Austria', 
            'CAN' : 'Canada',
            'CZE' : 'Czech Rep.',
            'FIN' : 'Finland',
            'GER' : 'Germany',
            'RUS' : 'Russia',
            'SLO' : 'Slovakia',
            'SWE' : 'Sweden',
            'SWI' : 'Switzerland'}

In [355]:
playerid = '348134'

user, password, server, database = load_db_credentials()
engine = create_engine(f'postgresql://{user}:{password}@{server}:5432/{database}')

In [356]:
df = pd.read_csv('../../apple-model-app/data/draft-value.csv')
teams = json.load(open('../../apple-model-app/assets/team-styles.json'))
info = pd.read_sql(f''' select * from player_info where catches is NULL ''', engine)

In [364]:
info['playerid'] = info['playerid'].astype(int)

In [412]:
fields = ['playerid', 'player_name', 'position', 'position_count', 'positions', 'team', 'nhl_likelihood',
          'shoots', 'date_of_birth', 'Projected Role', 'rights', 'nation', 'draft_round', 'draft_pick']

rosters = pd.read_csv('../../apple-model-app/data/wjc-rosters.csv')
rosters.rename(columns={'position' : 'positions', 'player' : 'player_name'}, inplace=True)

players = df.sort_values(by=['player', 'season_order'],
                                 ascending=[True, False])\
        .drop_duplicates(subset=['playerid'])\
        .loc[df.playerid.isin(rosters.playerid)]

rosters = rosters[['player_name', 'playerid', 'team', 'season_stage', 'positions']].merge(
    players,
    how='left',
    on='playerid'
).sort_values('nhl_likelihood', ascending=False)\
.merge(info[['playerid', 'shoots']],
       on='playerid',
       how='left'
)

rosters['nation'] = np.where(rosters.nation.isnull(),
                      rosters.team.map(countries),
                      rosters.nation)

rosters['nation_flag'] = rosters.team.map(countries).apply(lambda x : country_flag([x]))

rosters['position'] = np.where(rosters.position.isnull(),
                           rosters.positions.apply(get_primary_position),
                           rosters.position)

rosters['positions'] = rosters['positions'].apply(ast.literal_eval)

rosters['position_count'] = rosters.positions.apply(lambda x : len(x))

rosters.to_csv('../../apple-model-app/data/wjc-rosters.csv')


# get team roster page

In [413]:
rosters = pd.read_csv('../../apple-model-app/data/wjc-rosters.csv')

In [414]:
# 14 F and 8 D
team = 'USA'
team_roster = rosters[rosters.team == team]

left_w, centers, right_w = get_forward_ids(team_roster)
left_d, right_d = get_defence_ids(team_roster)

f_roster = dbc.Row(
      [
        dbc.Col([dbc.Row(get_roster_card(p), style={'height' : '20%'}) for p in left_w], width=4),
        dbc.Col([dbc.Row(get_roster_card(p), style={'height' : '20%'}) for p in centers], width=4),
        dbc.Col([dbc.Row(get_roster_card(p), style={'height' : '20%'}) for p in right_w], width=4)
      ],
      style={'width' : '100%', 'height' : '100%',}
        )

d_roster = dbc.Row(
      [
        dbc.Col([dbc.Row(card, style={'height' : '20%'}) for p in left_d], width=4),
        dbc.Col([dbc.Row(card, style={'height' : '20%'}) for p in right_d], width=4),
      ],
      style={'width' : '100%', 'height' : '100%',},
      justify = 'center',
        )

In [404]:
team_roster['nation_flag'] = team_roster.team.map(countries).apply(lambda x : country_flag([x]))

/Users/connorjung/.pyenv/versions/apple/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Player card Div

In [29]:
player = df[df.playerid == int(playerid)].sort_values('season_order', ascending=False).head(1)

logo = get_logo(player.rights.item())
dims = get_logo_dims(player.rights.item(), 40)

card = dbc.Card(
    [
        dbc.CardHeader(
            [dbc.Col(
                html.Img(id='draft-team', src=logo, **dims),
                    width=4,
                    align='top',
            ),
            dbc.Col(
                dcc.Markdown(f'''{player.player.item()}'''
                            ),
                            width=8,
                            align="top", 
                    )
            ]
        ),
        dbc.CardBody(
            [
                dbc.Col([
                    html.P('NHL %'),
                    html.P('NHL Role'),
                    html.P('Round'),
                    html.P('Pick'),
                ]),
                dbc.Col([
                    html.P(player.nhl_likelihood.item()),
                    html.P(player['Projected Role'].item()),
                    html.P(player.draft_round.item()),
                    html.P(player.draft_pick.item()),
                ])
            ]
        ),
        dbc.CardFooter(player.date_of_birth.str.split('-').item()[0]),
    ],
)